In [ ]:
import pickle

In [ ]:
tokenized_stores = {'en_train': [], 'en_dev': [], 'en_test': [], 'se_train': [], 'se_dev': [], 'se_test': []}

In [ ]:

for key in tokenized_stores:
    file_name = "/content/" + str(key)[3:] + "." + str(key)[0:2]
    load = open(file_name)
    sentences = load.read().split('\n')
    
    for sentence in sentences:
        token_store = sentence.split(' ')
        tokenized_stores[key].append(token_store)

In [ ]:
print(tokenized_stores['se_train'][2])

['He', 'quedado', 'conmovido', 'por', 'esta', 'conferencia,', 'y', 'deseo', 'agradecer', 'a', 'todos', 'ustedes', 'sus', 'amables', 'comentarios', 'acerca', 'de', 'lo', 'que', 'tenía', 'que', 'decir', 'la', 'otra', 'noche.', '']


In [ ]:
train_size = len(tokenized_stores['en_train'])
dev_size = len(tokenized_stores['en_dev'])
test_size = len(tokenized_stores['en_test'])

In [ ]:
en_words = {}
es_words = {}

for key in tokenized_stores:
    if str(key)[0] == 'e':
        # creating en_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in en_words:
                    en_words[word] += 1
                else:
                    en_words[word] = 1
    else:
        # creating es_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in es_words:
                    es_words[word] += 1
                else:
                    es_words[word] = 1
                    
en_vocab = len(en_words)
es_vocab = len(es_words)
print("Number of Unique Words:")
print("> English:", str(en_vocab))
print("> Spanish:", str(es_vocab))

Number of Unique Words:
> English: 48811
> Spanish: 61372


In [ ]:

t = {}

uniform = 1 / (en_vocab * es_vocab)

In [ ]:
n_iters = 0
max_iters = 25

fine_tune = 1
has_converged = False

while n_iters < max_iters and has_converged == False:
    has_converged = True
    max_change = -1

    n_iters += 1 
    count = {}
    total = {}
    for index in range(train_size):
        s_total = {}
        for en_word in tokenized_stores['en_train'][index]:
            s_total[en_word] = 0
            for es_word in tokenized_stores['se_train'][index]:
                if (en_word, es_word) not in t:
                    t[(en_word, es_word)] = uniform
                s_total[en_word] += t[(en_word, es_word)]

        for en_word in tokenized_stores['en_train'][index]:
            for es_word in tokenized_stores['se_train'][index]:
                if (en_word, es_word) not in count:
                    count[(en_word, es_word)] = 0
                count[(en_word, es_word)] += (t[(en_word, es_word)] / s_total[en_word])

                if es_word not in total:
                    total[es_word] = 0
                total[es_word] += (t[(en_word, es_word)] / s_total[en_word])



    if fine_tune == 0:
      updated = {}
   
      for index in range(train_size):
          for es_word in tokenized_stores['se_train'][index]:
              for en_word in tokenized_stores['en_train'][index]:
                  if (en_word, es_word) in updated:
                      continue
                  updated[(en_word, es_word)] = 1
                  if abs(t[(en_word, es_word)] - count[(en_word, es_word)] / total[es_word]) > 0.01:
                      has_converged = False
                      max_change = max(max_change, abs(t[(en_word, es_word)] - count[(en_word, es_word)] / total[es_word]))
                  t[(en_word, es_word)] = count[(en_word, es_word)] / total[es_word]

    elif fine_tune == 1:
 
      max_words = 1000
      n_es_words = 0
      updates = 0

      for es_word_tuples in sorted(es_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
          es_word = es_word_tuples[0]
          n_es_words += 1
          if n_es_words > max_words:
              break
          n_en_words = 0
          for en_word_tuples in sorted(en_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
              en_word = en_word_tuples[0]
              n_en_words += 1
              if n_en_words > max_words:
                  break
              if (en_word, es_word) not in count or es_word not in total:
                  continue
               
              else:
                  if abs(t[(en_word, es_word)] - count[(en_word, es_word)] / total[es_word]) > 0.005:
                      has_converged = False
                      max_change = max(max_change, abs(t[(en_word, es_word)] - count[(en_word, es_word)] / total[es_word]))
                  t[(en_word, es_word)] = count[(en_word, es_word)] / total[es_word]

    print("Iteration " + str(n_iters) + " Completed, Maximum Change: " + str(max_change))


Iteration 1 Completed, Maximum Change: 0.2768782008740783
Iteration 2 Completed, Maximum Change: 0.34932410271091824
Iteration 3 Completed, Maximum Change: 0.20264198358496976
Iteration 4 Completed, Maximum Change: 0.12245388811795044
Iteration 5 Completed, Maximum Change: 0.08040468373817894
Iteration 6 Completed, Maximum Change: 0.05104219402918564
Iteration 7 Completed, Maximum Change: 0.03274035743078252
Iteration 8 Completed, Maximum Change: 0.022929256490210936
Iteration 9 Completed, Maximum Change: 0.017229226299106948
Iteration 10 Completed, Maximum Change: 0.013231263522106351
Iteration 11 Completed, Maximum Change: 0.010354735163378592
Iteration 12 Completed, Maximum Change: 0.008236117038911805
Iteration 13 Completed, Maximum Change: 0.006643797653858252
Iteration 14 Completed, Maximum Change: 0.005425935978163321
Iteration 15 Completed, Maximum Change: 0.005217421559786033
Iteration 16 Completed, Maximum Change: -1


In [ ]:

limit = 40
for element in sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True):
  print(element)
  limit -= 1
  if limit <= 0:
    break

(('(Applause)', '(Aplausos)'), 0.70931401913795)
(('and', 'y'), 0.686886867612092)
(('(Laughter)', '(Risas)'), 0.6656162257326975)
(('because', 'porque'), 0.6373236283209089)
(('But', 'Pero'), 0.6182611233955007)
(('this', 'este'), 0.6182330882236168)
(('this', 'esta'), 0.6162898306837068)
(('or', 'o'), 0.6148935231586057)
(('--', '--'), 0.6059776444937575)
(('but', 'pero'), 0.6040636838090476)
(('', ''), 0.6027015939365593)
(('three', 'tres'), 0.601321812204926)
(('♫', '♫'), 0.5985622594322169)
(('our', 'nuestra'), 0.5956485026013792)
(('our', 'nuestros'), 0.5869222226729975)
(('where', 'donde'), 0.5866110704386672)
(('And', 'Y'), 0.5859161307430665)
(('my', 'mi'), 0.5848632750735021)
(('when', 'cuando'), 0.5841553343362785)
(('two', 'dos'), 0.580109468799991)
(('first', 'primera'), 0.5798809516786032)
(('different', 'diferentes'), 0.574779831500915)
(('our', 'nuestro'), 0.5747460125516214)
(('a', 'un'), 0.5710174555832708)
(('my', 'mis'), 0.5677954375973354)
(('these', 'estas'), 0.56

In [ ]:

file = open("translation_model.pkl","wb")
pickle.dump(t, file)
file.close()

In [ ]:

model_name = "translation_model.pkl"
pickle_in = open(model_name,"rb")
t = pickle.load(pickle_in)

In [ ]:
I = {}
for index in range(train_size):
    for en_id in range(len(tokenized_stores['en_train'][index])):
        length = len(tokenized_stores['en_train'][index])
        if length not in I:
            I[length] = {} # maps the positional difference to a tuple: (sum of t's, count)
        for es_id in range(len(tokenized_stores['se_train'][index])):
            if (es_id - en_id) not in I[length]:
                I[length][(es_id - en_id)] = [t[(tokenized_stores['en_train'][index][en_id], tokenized_stores['se_train'][index][es_id])], 1]
            else:
                I[length][(es_id - en_id)][0] += t[(tokenized_stores['en_train'][index][en_id], tokenized_stores['se_train'][index][es_id])]
                I[length][(es_id - en_id)][1] += 1

In [ ]:

sentence_lengths = []
for key in I.keys():
    if key not in sentence_lengths:
        sentence_lengths.append(key)
sentence_lengths.sort()
print(sentence_lengths)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 88, 89, 90, 91, 95, 96, 97, 104, 105, 108, 121, 122, 125, 126, 171, 179, 240, 311, 335, 352, 449]


In [ ]:

 
p = {}
for key in I.keys():
    p[key] = {}
    sum_val = 0
    for diff in I[key].keys():
        p[key][diff] = I[key][diff][0] / I[key][diff][1]
        sum_val += p[key][diff]
    for diff in p[key].keys():
        p[key][diff] /= sum_val

In [ ]:
for index in range(train_size):
    length_en = len(tokenized_stores['en_train'][index])
    length_es = len(tokenized_stores['se_train'][index])
    if length_es - length_en > 10 and length_en == 1:
        print("Length of English Sentence:", str(length_en))
        print("Length of Spanish Sentence:", str(length_es))

In [ ]:

init = {}
for length in p:
    max_prob = -1
    max_jump = 0
    for key in p[length].keys():
        if p[length][key] > max_prob:
            max_prob = p[length][key]
            max_jump = key
    init[length] = max_jump

In [ ]:
!pip install nltk

In [ ]:

bigrams = {}
unigrams = {}


def model(dataset_size, dataset_name):
    global bigrams
    global unigrams
    for index in range(dataset_size):
        token_A = ''
        for es_token in tokenized_stores[dataset_name][index]:
            if es_token not in unigrams:
                unigrams[es_token] = 1
            else:
                unigrams[es_token] += 1
            
            token_B = es_token
            if (token_A, token_B) not in bigrams:
                bigrams[(token_A, token_B)] = 1
            else:
                bigrams[(token_A, token_B)] += 1
            token_A = token_B

model(train_size, 'se_train')
model(dev_size, 'se_dev')

bigram_count = len(bigrams)
unigram_count = len(unigrams)
print("Number of Unique Bigrams:", bigram_count)
print("Number of Unique Unigrams:", unigram_count)

Number of Unique Bigrams: 224510
Number of Unique Unigrams: 54018


In [ ]:
from itertools import permutations
import nltk

computed_sentences = []
total_BLEU = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 7: 0}
null_BLEU_count = 0

sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def find_translation(en_token):
    for element in sorted_t:
        if element[0][0].lower() == en_token:
            return element[0][1]
    return ""

def get_prob(seq):
    if len(seq) < 2:
        return 1
    score = 0
    token_A = ''
    for es_token in seq:
        token_B = es_token
        if (token_A, token_B) not in bigrams:
            if token_B not in unigrams:
                continue
            else:
                score += unigrams[token_B] / unigram_count
        else:
            base_token_count = 0
            if token_A in unigrams:
                base_token_count = unigrams[token_A]
            score += (bigrams[(token_A, token_B)] + 1) / (base_token_count + unigram_count)
        token_A = token_B
    return score

count = 0
for index in range(test_size):
    if len(tokenized_stores['en_test'][index]) > 8 or len(tokenized_stores['en_test'][index]) < 2:
        continue

    translated_words = []
    for en_token in tokenized_stores['en_test'][index]:
        translation = find_translation(en_token)
        if translation != "":
            translated_words.append(translation)

    perm = permutations(translated_words)

    best_seq = translated_words
    best_prob = -1

    for seq in perm:
        prob = get_prob(seq)
        if prob > best_prob:
            best_prob = prob
            best_seq = seq

    BLEU_scores = []
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['se_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['se_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method2))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['se_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method3))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['se_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['se_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method5))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['se_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method7))

    for key in total_BLEU.keys():
        if key == 7:
            consider = 5
        else: consider = key - 1
        total_BLEU[key] += BLEU_scores[consider]
    
    if BLEU_scores[0] == 0:
        null_BLEU_count += 1
    
    count += 1
    print("Sentence Index: ", str(count))
    print("English Sentence:", str(tokenized_stores['en_test'][index]))
    print("Reference Hindi Sentence:", str(tokenized_stores['se_test'][index]))
    print("Translated Sentence:", str(best_seq))
    print("Translation BLEU Scores", str(BLEU_scores))
    print()
    
    computed_sentences.append([tokenized_stores['en_test'][index], tokenized_stores['se_test'][index], best_seq, BLEU_scores])

tested = count

Sentence Index:  1
English Sentence: ['At', 'least', 'they', 'say', 'so.']
Reference Spanish Sentence: ['Al', 'menos,', 'eso', 'dicen.']
Translated Sentence: ('menos', 'tenían', 'decir', '¿cómo')
Translation BLEU Scores [0.019229913610012243, 0.10063351655856652, 0.03823246852690465, 0.11593116128936173, 0.04934931901860593, 0.14898359094953884]

Sentence Index:  2
English Sentence: ['They', 'make', 'teeth', 'dirty', 'and', 'breath', 'stinky.']
Reference Spanish Sentence: ['Ellas', 'hacen', 'dientes', 'sucias', 'y', 'aliento', 'malolientes.']
Translated Sentence: ('hacen', 'dientes', 'sucias', 'y', 'aliento', '¿cómo')
Translation BLEU Scores [0.045131921809482646, 0.1878296463217631, 0.08973024087021203, 0.10990031779775743, 0.06060465697360007, 0.14348435345097704]

Sentence Index:  3
English Sentence: ['Does', 'kalajar', 'occur', 'because', 'of', 'sun', '.']
Reference Spanish Sentence: ['Hace', 'kalajar', 'ocurre', 'porque', 'de', 'sol', '.']
Translated Sentence: ( 'ocurre', 'porque'

In [ ]:
# Results:
import statistics
print("Number of Samples Tested Upon: " + str(tested))
print()

print("Average BLEU Score using Various Smoothing Functions (considering all test samples)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / tested))
print()
print("Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / (tested - null_BLEU_count)))

Number of Samples Tested Upon: 50

Average BLEU Score using Various Smoothing Functions (considering all test samples)
Method 1: 0.05145334190563911
Method 2: 0.18090274652264405
Method 3: 0.09042839674590069
Method 4: 0.15545775044775517
Method 5: 0.04580429833529106
Method 7: 0.2214522086114808

Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)
Method 1: 0.0520266422545892
Method 2: 0.1817374964516776
Method 3: 0.0945886522552555
Method 4: 0.15863035759975017
Method 5: 0.04568921445511668
Method 7: 0.22122254684268226
